# Curadoria de Dados em SQL - Base Analítica de E-commerce

Este notebook documenta o processo completo de **curadoria e padronização de dados** utilizado como base analítica para múltiplas Provas de Conceito (POCs) de Analytics e BI.

O foco aqui **não é análise de negócio**, mas sim a construção de uma **camada de dados confiável, auditável e reutilizável**, preparada para consumo analítico posterior.

---

## Contexto do Dataset

O conjunto de dados original utilizado é o [**Olist Brazilian E-Commerce Dataset**](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce), disponibilizado publicamente no Kaggle:

Esse dataset representa transações reais de um marketplace brasileiro, contendo informações sobre pedidos, itens, pagamentos, clientes, sellers, produtos, geolocalização e avaliações de clientes.

---

## Escopo deste Notebook

Neste notebook, são realizadas as seguintes etapas:

- Conexão com o banco de dados analítico em DuckDB
- Padronização e renomeação das tabelas originais
- Inspeção de qualidade dos dados (unicidade, nulos, domínio)
- Criação de **views curadas (`curated_*`)**
- Deduplicação e agregações estratégicas (1 linha por pedido)
- Validação estrutural e estatística das views geradas

O resultado final é uma **camada Silver / Curated**, pronta para servir de base para diferentes produtos analíticos (BI executivo, análises diagnósticas, EDA).

---

## Nota sobre a Ingestão dos Dados

O processo de ingestão e conversão dos arquivos CSV originais para o formato DuckDB foi realizado previamente, por meio de um [**script dedicado de ingestão**](scripts/ingest_olist_to_duckdb.py).

Este notebook assume que:
- Os dados brutos já foram convertidos para DuckDB
- As tabelas originais refletem fielmente o conteúdo do dataset do Kaggle
- A curadoria começa a partir dessa base estruturada

---

## Organização do Notebook

A partir da próxima seção, o notebook segue uma estrutura padronizada por entidade,
composta por:

1. Inspeção inicial dos dados
2. Construção da view curada
3. Validação pós-curadoria

# Conexão e Verificação Inicial do Ambiente Analítico

Nesta seção são realizadas a importação das bibliotecas necessárias, a conexão com o banco de dados DuckDB e uma verificação inicial das tabelas disponíveis, garantindo que o ambiente esteja consistente antes do início da curadoria por entidade.


In [1]:
import sys; sys.path.insert(0, "..")
from src.paths import PROCESSED_DATA
import duckdb

In [2]:
con = duckdb.connect(database=str(PROCESSED_DATA), read_only=False)
con.execute("SELECT 1").fetchone()

(1,)

In [3]:
# listar tabelas e views
con.execute("""
SELECT table_schema, table_name, table_type
FROM information_schema.tables
WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
ORDER BY table_type, table_schema, table_name;
""").df()

,table_schema,table_name,table_type
0,main,olist_customers_dataset,BASE TABLE
1,main,olist_geolocation_dataset,BASE TABLE
2,main,olist_order_items_dataset,BASE TABLE
3,main,olist_order_payments_dataset,BASE TABLE
4,main,olist_order_reviews_dataset,BASE TABLE
5,main,olist_orders_dataset,BASE TABLE
6,main,olist_products_dataset,BASE TABLE
7,main,olist_sellers_dataset,BASE TABLE
8,main,product_category_name_translation,BASE TABLE


In [4]:
rename_statements = [
    "ALTER TABLE olist_orders_dataset RENAME TO orders;",
    "ALTER TABLE olist_order_items_dataset RENAME TO order_items;",
    "ALTER TABLE olist_order_payments_dataset RENAME TO order_payments;",
    "ALTER TABLE olist_order_reviews_dataset RENAME TO order_reviews;",
    "ALTER TABLE olist_customers_dataset RENAME TO customers;",
    "ALTER TABLE olist_sellers_dataset RENAME TO sellers;",
    "ALTER TABLE olist_products_dataset RENAME TO products;",
    "ALTER TABLE olist_geolocation_dataset RENAME TO geolocation;",
    "ALTER TABLE product_category_name_translation RENAME TO category_translation;"
]

for stmt in rename_statements:
    con.execute(stmt)


In [5]:
con.execute("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'main'
ORDER BY table_name;
""").df()

,table_name
0,category_translation
1,customers
2,geolocation
3,order_items
4,order_payments
5,order_reviews
6,orders
7,products
8,sellers


# TABLE `orders`

In [6]:
# Inspeção do schema (SQL)
con.execute("DESCRIBE orders;").df()

,column_name,column_type,null,key,default,extra
0,order_id,VARCHAR,YES,None,None,None
1,customer_id,VARCHAR,YES,None,None,None
2,order_status,VARCHAR,YES,None,None,None
3,order_purchase_timestamp,TIMESTAMP,YES,None,None,None
4,order_approved_at,TIMESTAMP,YES,None,None,None
5,order_delivered_carrier_date,TIMESTAMP,YES,None,None,None
6,order_delivered_customer_date,TIMESTAMP,YES,None,None,None
7,order_estimated_delivery_date,TIMESTAMP,YES,None,None,None


## Inspeção inicial (qualidade e sanidade)

In [7]:
# Integridade de chave
con.execute("""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT order_id) AS distinct_order_id,
  COUNT(*) - COUNT(DISTINCT order_id) AS dup_order_id
FROM orders;
""").df()

,total_rows,distinct_order_id,dup_order_id
0,99441,99441,0


In [8]:
# Nulos críticos
con.execute("""
SELECT
  SUM(CASE WHEN order_id IS NULL THEN 1 ELSE 0 END) AS null_order_id,
  SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) AS null_customer_id,
  SUM(CASE WHEN order_status IS NULL THEN 1 ELSE 0 END) AS null_order_status
FROM orders;
""").df()

,null_order_id,null_customer_id,null_order_status
0,0.0,0.0,0.0


In [9]:
# Sanidade do domínio (Distribuição de status)
con.execute("""
SELECT order_status, COUNT(*) AS n
FROM orders
GROUP BY 1
ORDER BY n DESC;
""").df()

,order_status,n
0,delivered,96478
1,shipped,1107
2,canceled,625
3,unavailable,609
4,invoiced,314
5,processing,301
6,created,5
7,approved,2


## Criar view `curated_orders`

In [10]:
con.execute("""
CREATE OR REPLACE VIEW curated_orders AS
WITH base AS (
  SELECT
    order_id,
    customer_id,
    LOWER(TRIM(order_status)) AS order_status,

    order_purchase_timestamp AS order_purchase_ts,
    order_approved_at AS order_approved_ts,
    order_delivered_carrier_date AS order_delivered_carrier_ts,
    order_delivered_customer_date AS order_delivered_customer_ts,
    order_estimated_delivery_date AS order_estimated_delivery_ts,

    CAST(order_purchase_timestamp AS DATE) AS purchase_date,
    CAST(order_delivered_customer_date AS DATE) AS delivered_date,
    CAST(order_estimated_delivery_date AS DATE) AS estimated_delivery_date
  FROM orders
),
calc AS (
  SELECT
    *,
    DATE_DIFF('day', purchase_date, delivered_date) AS days_to_deliver,
    -- signed: positivo = atrasou, negativo = adiantou
    DATE_DIFF('day', estimated_delivery_date, delivered_date) AS delay_days
  FROM base
)
SELECT
  *,
  -- positivos “prontos pra BI”
  CASE WHEN delay_days > 0 THEN delay_days ELSE 0 END AS delay_days_pos,
  CASE WHEN delay_days < 0 THEN -delay_days ELSE 0 END AS early_days_pos,

  CASE WHEN order_status = 'delivered' THEN 1 ELSE 0 END AS is_delivered,
  CASE
    WHEN delivered_date IS NULL OR estimated_delivery_date IS NULL THEN 0
    WHEN delay_days > 0 THEN 1
    ELSE 0
  END AS is_delayed
FROM calc;
""")

## Validar `curated_orders`

In [11]:
# Schema da view
con.execute("DESCRIBE curated_orders;").df()

,column_name,column_type,null,key,default,extra
0,order_id,VARCHAR,YES,None,None,None
1,customer_id,VARCHAR,YES,None,None,None
2,order_status,VARCHAR,YES,None,None,None
3,order_purchase_ts,TIMESTAMP,YES,None,None,None
4,order_approved_ts,TIMESTAMP,YES,None,None,None
5,order_delivered_carrier_ts,TIMESTAMP,YES,None,None,None
6,order_delivered_customer_ts,TIMESTAMP,YES,None,None,None
7,order_estimated_delivery_ts,TIMESTAMP,YES,None,None,None
8,purchase_date,DATE,YES,None,None,None
9,delivered_date,DATE,YES,None,None,None


In [12]:
# Estatísticas/sanity
con.execute("""
SELECT
  COUNT(*) AS delivered,
  SUM(CASE WHEN delay_days_pos > 0 THEN 1 ELSE 0 END) AS delayed_delivered,
  AVG(CASE WHEN delay_days_pos > 0 THEN delay_days_pos END) AS avg_delay_when_delayed,
  AVG(CASE WHEN early_days_pos > 0 THEN early_days_pos END) AS avg_early_when_early
FROM curated_orders
WHERE is_delivered = 1;
""").df()

,delivered,delayed_delivered,avg_delay_when_delayed,avg_early_when_early
0,96478,6534.0,10.620141,13.707177


In [13]:
# Integridade (unicidade / duplicidade)
con.execute("""
SELECT
  MIN(delay_days) AS min_signed_delay,
  MAX(delay_days) AS max_signed_delay,
  MAX(delay_days_pos) AS max_delay_pos,
  MAX(early_days_pos) AS max_early_pos
FROM curated_orders
WHERE is_delivered = 1;
""").df()


,min_signed_delay,max_signed_delay,max_delay_pos,max_early_pos
0,-147,188,188,147


# TABLE `ordem_items`

In [14]:
# Inspeção do schema (SQL)
con.execute("DESCRIBE order_items;").df()

,column_name,column_type,null,key,default,extra
0,order_id,VARCHAR,YES,None,None,None
1,order_item_id,BIGINT,YES,None,None,None
2,product_id,VARCHAR,YES,None,None,None
3,seller_id,VARCHAR,YES,None,None,None
4,shipping_limit_date,TIMESTAMP,YES,None,None,None
5,price,DOUBLE,YES,None,None,None
6,freight_value,DOUBLE,YES,None,None,None


## Inspeção inicial (qualidade e sanidade)

In [15]:
# Integridade de chave (Chave composta)
con.execute("""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT order_id || '-' || CAST(order_item_id AS VARCHAR)) AS distinct_key,
  COUNT(*) - COUNT(DISTINCT order_id || '-' || CAST(order_item_id AS VARCHAR)) AS dup_key
FROM order_items;
""").df()

,total_rows,distinct_key,dup_key
0,112650,112650,0


In [16]:
# Nulos críticos
con.execute("""
SELECT
  SUM(order_id IS NULL) AS null_order_id,
  SUM(order_item_id IS NULL) AS null_order_item_id,
  SUM(product_id IS NULL) AS null_product_id,
  SUM(seller_id IS NULL) AS null_seller_id,
  SUM(price IS NULL) AS null_price,
  SUM(freight_value IS NULL) AS null_freight
FROM order_items;
""").df()

,null_order_id,null_order_item_id,null_product_id,null_seller_id,null_price,null_freight
0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Sanidade do domínio (Valores negativos)
con.execute("""
SELECT
  SUM(CASE WHEN price < 0 THEN 1 ELSE 0 END) AS neg_price,
  SUM(CASE WHEN freight_value < 0 THEN 1 ELSE 0 END) AS neg_freight
FROM order_items;
""").df()


,neg_price,neg_freight
0,0.0,0.0


## Criar a view `curated_order_items`

In [18]:
con.execute("""
CREATE OR REPLACE VIEW curated_order_items AS
SELECT
  TRIM(order_id) AS order_id,
  CAST(order_item_id AS BIGINT) AS order_item_id,
  TRIM(product_id) AS product_id,
  TRIM(seller_id) AS seller_id,

  shipping_limit_date AS shipping_limit_ts,
  CAST(shipping_limit_date AS DATE) AS shipping_limit_date,

  -- valores (clamp para evitar lixo)
  CASE WHEN price IS NULL THEN NULL WHEN price < 0 THEN 0 ELSE price END AS price,
  CASE WHEN freight_value IS NULL THEN NULL WHEN freight_value < 0 THEN 0 ELSE freight_value END AS freight_value,

  -- métricas por item
  (CASE WHEN price IS NULL THEN 0 WHEN price < 0 THEN 0 ELSE price END)
  + (CASE WHEN freight_value IS NULL THEN 0 WHEN freight_value < 0 THEN 0 ELSE freight_value END)
  AS item_gmv

FROM order_items;
""")

### Validar `curated_order_items`

In [19]:
# Schema view
con.execute("DESCRIBE curated_order_items;").df()

,column_name,column_type,null,key,default,extra
0,order_id,VARCHAR,YES,None,None,None
1,order_item_id,BIGINT,YES,None,None,None
2,product_id,VARCHAR,YES,None,None,None
3,seller_id,VARCHAR,YES,None,None,None
4,shipping_limit_ts,TIMESTAMP,YES,None,None,None
5,shipping_limit_date,DATE,YES,None,None,None
6,price,DOUBLE,YES,None,None,None
7,freight_value,DOUBLE,YES,None,None,None
8,item_gmv,DOUBLE,YES,None,None,None


In [20]:
# Estatísticas/sanity
con.execute("""
SELECT
  COUNT(*) AS rows,
  SUM(item_gmv) AS total_item_gmv,
  AVG(item_gmv) AS avg_item_gmv,
  MIN(item_gmv) AS min_item_gmv,
  MAX(item_gmv) AS max_item_gmv
FROM curated_order_items;
""").df()

,rows,total_item_gmv,avg_item_gmv,min_item_gmv,max_item_gmv
0,112650,1.584355e+07,140.644059,6.08,6929.31


In [21]:
# Integridade (unicidade / duplicidade)
con.execute("""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT order_id || '-' || CAST(order_item_id AS VARCHAR)) AS distinct_key,
  COUNT(*) - COUNT(DISTINCT order_id || '-' || CAST(order_item_id AS VARCHAR)) AS dup_key
FROM curated_order_items;
""").df()


,total_rows,distinct_key,dup_key
0,112650,112650,0


# TABLE `order_payments`

In [22]:
# Inspeção do schema (SQL)
con.execute("DESCRIBE order_payments;").df()

,column_name,column_type,null,key,default,extra
0,order_id,VARCHAR,YES,None,None,None
1,payment_sequential,BIGINT,YES,None,None,None
2,payment_type,VARCHAR,YES,None,None,None
3,payment_installments,BIGINT,YES,None,None,None
4,payment_value,DOUBLE,YES,None,None,None


## Inspeção inicial (qualidade e sanidade)

In [23]:
# Integridade de chave (Chave composta)
con.execute("""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT order_id || '-' || CAST(payment_sequential AS VARCHAR)) AS distinct_key,
  COUNT(*) - COUNT(DISTINCT order_id || '-' || CAST(payment_sequential AS VARCHAR)) AS dup_key
FROM order_payments;
""").df()

,total_rows,distinct_key,dup_key
0,103886,103886,0


In [24]:
# Nulos críticos
con.execute("""
SELECT
  SUM(order_id IS NULL) AS null_order_id,
  SUM(payment_type IS NULL) AS null_payment_type,
  SUM(payment_installments IS NULL) AS null_installments,
  SUM(payment_value IS NULL) AS null_payment_value
FROM order_payments;
""").df()

,null_order_id,null_payment_type,null_installments,null_payment_value
0,0.0,0.0,0.0,0.0


In [25]:
# Sanidade do domínio (Valores inválidos)
con.execute("""
SELECT
  SUM(CASE WHEN payment_value < 0 THEN 1 ELSE 0 END) AS neg_payment_value,
  SUM(CASE WHEN payment_installments <= 0 THEN 1 ELSE 0 END) AS non_positive_installments
FROM order_payments;
""").df()

,neg_payment_value,non_positive_installments
0,0.0,2.0


## Criar view `curated_order_payments`

In [26]:
con.execute("""
CREATE OR REPLACE VIEW curated_order_payments AS
SELECT
  TRIM(order_id) AS order_id,
  CAST(payment_sequential AS BIGINT) AS payment_sequential,
  LOWER(TRIM(payment_type)) AS payment_type,

  CASE
    WHEN payment_installments IS NULL THEN NULL
    WHEN payment_installments < 1 THEN 1
    ELSE payment_installments
  END AS payment_installments,

  CASE
    WHEN payment_value IS NULL THEN NULL
    WHEN payment_value < 0 THEN 0
    ELSE payment_value
  END AS payment_value

FROM order_payments;
""")


### Validar `ordem_payments`

In [27]:
# Schema view
con.execute("DESCRIBE curated_order_payments;").df()

,column_name,column_type,null,key,default,extra
0,order_id,VARCHAR,YES,None,None,None
1,payment_sequential,BIGINT,YES,None,None,None
2,payment_type,VARCHAR,YES,None,None,None
3,payment_installments,BIGINT,YES,None,None,None
4,payment_value,DOUBLE,YES,None,None,None


In [28]:
# Estatíticas/sanity
con.execute("""
SELECT
  COUNT(*) AS rows,
  SUM(payment_value) AS total_payment_value,
  AVG(payment_value) AS avg_payment_value,
  MAX(payment_installments) AS max_installments
FROM curated_order_payments;
""").df()


,rows,total_payment_value,avg_payment_value,max_installments
0,103886,1.600887e+07,154.10038,24


In [29]:
# Integridade (unicidade / duplicidade)
con.execute("""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT order_id || '-' || CAST(payment_sequential AS VARCHAR)) AS distinct_key,
  COUNT(*) - COUNT(DISTINCT order_id || '-' || CAST(payment_sequential AS VARCHAR)) AS dup_key
FROM curated_order_payments;
""").df()


,total_rows,distinct_key,dup_key
0,103886,103886,0


## Criar view `payments_by_order`

In [30]:
con.execute("""
CREATE OR REPLACE VIEW curated_payments_by_order AS
WITH agg AS (
  SELECT
    order_id,
    SUM(payment_value) AS total_payment_value,
    MAX(payment_installments) AS max_installments,
    COUNT(*) AS num_payments
  FROM curated_order_payments
  GROUP BY 1
),
ranked AS (
  SELECT
    order_id,
    payment_type,
    SUM(payment_value) AS payment_type_value,
    ROW_NUMBER() OVER (
      PARTITION BY order_id
      ORDER BY SUM(payment_value) DESC, payment_type
    ) AS rn
  FROM curated_order_payments
  GROUP BY 1,2
)
SELECT
  a.order_id,
  a.total_payment_value,
  a.max_installments,
  a.num_payments,
  r.payment_type AS payment_type_main
FROM agg a
LEFT JOIN ranked r
  ON a.order_id = r.order_id AND r.rn = 1;
""")

### Validar `curated_payments_by_order`

In [31]:
# Schema view
con.execute("DESCRIBE curated_payments_by_order;").df()

,column_name,column_type,null,key,default,extra
0,order_id,VARCHAR,YES,None,None,None
1,total_payment_value,DOUBLE,YES,None,None,None
2,max_installments,BIGINT,YES,None,None,None
3,num_payments,BIGINT,YES,None,None,None
4,payment_type_main,VARCHAR,YES,None,None,None


In [32]:
# Estatísticas/sanity
con.execute("""
SELECT
  COUNT(*) AS orders_with_payment,
  AVG(total_payment_value) AS avg_payment_total,
  MAX(total_payment_value) AS max_payment_total,
  MAX(max_installments) AS max_installments,
  AVG(num_payments) AS avg_num_payments
FROM curated_payments_by_order;
""").df()

,orders_with_payment,avg_payment_total,max_payment_total,max_installments,avg_num_payments
0,99440,160.990267,13664.08,24,1.04471


In [33]:
# Integridade (unicidade / duplicidade)
con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT order_id) AS distinct_orders,
  COUNT(*) - COUNT(DISTINCT order_id) AS dup_orders
FROM curated_payments_by_order;
""").df()

,rows,distinct_orders,dup_orders
0,99440,99440,0


## Criar view `curated_items_by_order`

In [34]:
con.execute("""
CREATE OR REPLACE VIEW curated_items_by_order AS
SELECT
  order_id,
  COUNT(*) AS items_count,
  COUNT(DISTINCT product_id) AS unique_products,
  COUNT(DISTINCT seller_id) AS unique_sellers,

  SUM(price) AS total_price,
  SUM(freight_value) AS total_freight,
  SUM(item_gmv) AS total_gmv
FROM curated_order_items
GROUP BY 1;
""")


### Validar `curated_items_by_order`

In [35]:
# Schema view
con.execute("DESCRIBE curated_items_by_order;").df()

,column_name,column_type,null,key,default,extra
0,order_id,VARCHAR,YES,None,None,None
1,items_count,BIGINT,YES,None,None,None
2,unique_products,BIGINT,YES,None,None,None
3,unique_sellers,BIGINT,YES,None,None,None
4,total_price,DOUBLE,YES,None,None,None
5,total_freight,DOUBLE,YES,None,None,None
6,total_gmv,DOUBLE,YES,None,None,None


In [36]:
# Estatísticas/sanity
con.execute("""
SELECT
  AVG(items_count) AS avg_items_per_order,
  MAX(items_count) AS max_items_per_order,
  AVG(total_gmv) AS avg_gmv_per_order,
  MAX(total_gmv) AS max_gmv_per_order
FROM curated_items_by_order;
""").df()

,avg_items_per_order,max_items_per_order,avg_gmv_per_order,max_gmv_per_order
0,1.141731,21,160.577638,13664.08


In [37]:
# Integridade (unicidade / duplicidade)
con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT order_id) AS distinct_orders,
  COUNT(*) - COUNT(DISTINCT order_id) AS dup_orders
FROM curated_items_by_order;
""").df()

,rows,distinct_orders,dup_orders
0,98666,98666,0


# TABLE `customers`

In [38]:
# Inspeção do schema (SQL)
con.execute("DESCRIBE customers;").df()

,column_name,column_type,null,key,default,extra
0,customer_id,VARCHAR,YES,None,None,None
1,customer_unique_id,VARCHAR,YES,None,None,None
2,customer_zip_code_prefix,VARCHAR,YES,None,None,None
3,customer_city,VARCHAR,YES,None,None,None
4,customer_state,VARCHAR,YES,None,None,None


## Inspeção inicial (qualidade e sanidade)

In [39]:
# Integridade de chave ('customer_id')
con.execute("""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT customer_id) AS distinct_customer_id,
  COUNT(*) - COUNT(DISTINCT customer_id) AS dup_customer_id
FROM customers;
""").df()

,total_rows,distinct_customer_id,dup_customer_id
0,99441,99441,0


In [40]:
# Nulos críticos
con.execute("""
SELECT
  SUM(customer_id IS NULL) AS null_customer_id,
  SUM(customer_unique_id IS NULL) AS null_unique_id,
  SUM(customer_city IS NULL) AS null_city,
  SUM(customer_state IS NULL) AS null_state
FROM customers;
""").df()

,null_customer_id,null_unique_id,null_city,null_state
0,0.0,0.0,0.0,0.0


In [41]:
# Sanidade do domínio
con.execute("""
SELECT
  customer_state,
  COUNT(*) AS n
FROM customers
GROUP BY 1
ORDER BY n DESC;
""").df()

,customer_state,n
0,SP,41746
1,RJ,12852
2,MG,11635
3,RS,5466
4,PR,5045
5,SC,3637
6,BA,3380
7,DF,2140
8,ES,2033
9,GO,2020


## Criar view `curated_customers`

In [42]:
con.execute("""
CREATE OR REPLACE VIEW curated_customers AS
SELECT
  TRIM(customer_id) AS customer_id,
  TRIM(customer_unique_id) AS customer_unique_id,
  TRIM(customer_zip_code_prefix) AS customer_zip_code_prefix,
  LOWER(TRIM(customer_city)) AS customer_city,
  UPPER(TRIM(customer_state)) AS customer_state
FROM customers;
""")

### validar `curated_customers`

In [43]:
#Schema view
con.execute("DESCRIBE curated_customers;").df()

,column_name,column_type,null,key,default,extra
0,customer_id,VARCHAR,YES,None,None,None
1,customer_unique_id,VARCHAR,YES,None,None,None
2,customer_zip_code_prefix,VARCHAR,YES,None,None,None
3,customer_city,VARCHAR,YES,None,None,None
4,customer_state,VARCHAR,YES,None,None,None


In [44]:
# Estatísticas/sanity (UF fora do padrão)
con.execute("""
SELECT
  COUNT(*) AS total_rows,
  SUM(CASE WHEN customer_state IS NULL THEN 1 ELSE 0 END) AS null_state,
  SUM(CASE WHEN LENGTH(customer_state) <> 2 THEN 1 ELSE 0 END) AS invalid_len_state
FROM curated_customers;
""").df()

,total_rows,null_state,invalid_len_state
0,99441,0.0,0.0


In [45]:
# Estatísticas/sanity (Top cidades)
con.execute("""
SELECT customer_city, customer_state, COUNT(*) AS n
FROM curated_customers
GROUP BY 1,2
ORDER BY n DESC
LIMIT 15;
""").df()

,customer_city,customer_state,n
0,sao paulo,SP,15540
1,rio de janeiro,RJ,6882
2,belo horizonte,MG,2773
3,brasilia,DF,2131
4,curitiba,PR,1521
5,campinas,SP,1444
6,porto alegre,RS,1379
7,salvador,BA,1245
8,guarulhos,SP,1189
9,sao bernardo do campo,SP,938


# TABLE `geolocation`

In [46]:
# Inspeção do schema (SQL)
con.execute("DESCRIBE geolocation;").df()

,column_name,column_type,null,key,default,extra
0,geolocation_zip_code_prefix,VARCHAR,YES,None,None,None
1,geolocation_lat,DOUBLE,YES,None,None,None
2,geolocation_lng,DOUBLE,YES,None,None,None
3,geolocation_city,VARCHAR,YES,None,None,None
4,geolocation_state,VARCHAR,YES,None,None,None


## Inspeção inicial (qualidade e sanidade)

In [47]:
# Linhas por CEP
con.execute("""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT geolocation_zip_code_prefix) AS distinct_zip_prefix
FROM geolocation;
""").df()

,total_rows,distinct_zip_prefix
0,1000163,19015


In [48]:
# Nulos críticos
con.execute("""
SELECT
  SUM(geolocation_zip_code_prefix IS NULL) AS null_zip,
  SUM(geolocation_lat IS NULL) AS null_lat,
  SUM(geolocation_lng IS NULL) AS null_lng,
  SUM(geolocation_state IS NULL) AS null_state
FROM geolocation;
""").df()


,null_zip,null_lat,null_lng,null_state
0,0.0,0.0,0.0,0.0


In [49]:
# Latitude/Longitude fora do Brasil
con.execute("""
SELECT
  SUM(CASE WHEN geolocation_lat NOT BETWEEN -35 AND 5 THEN 1 ELSE 0 END) AS invalid_lat,
  SUM(CASE WHEN geolocation_lng NOT BETWEEN -75 AND -30 THEN 1 ELSE 0 END) AS invalid_lng
FROM geolocation;
""").df()


,invalid_lat,invalid_lng
0,29.0,26.0


## Criar view `curated_geolocation`

In [50]:
con.execute("""
CREATE OR REPLACE VIEW curated_geolocation AS
SELECT
  TRIM(geolocation_zip_code_prefix) AS zip_code_prefix,
  geolocation_lat AS latitude,
  geolocation_lng AS longitude,
  LOWER(TRIM(geolocation_city)) AS city,
  UPPER(TRIM(geolocation_state)) AS state
FROM geolocation
WHERE
  geolocation_lat BETWEEN -35 AND 5
  AND geolocation_lng BETWEEN -75 AND -30;
""")

## Criar view`curated_geolocation_by_zip`

In [51]:
con.execute("""
CREATE OR REPLACE VIEW curated_geolocation_by_zip AS
SELECT
  zip_code_prefix,
  AVG(latitude) AS latitude,
  AVG(longitude) AS longitude,
  MAX(state) AS state
FROM curated_geolocation
GROUP BY 1;
""")

### Validar `curated_geolocation_by_zip`

In [52]:
# Inspeção do schema (SQL)
con.execute("DESCRIBE curated_geolocation_by_zip;").df()

,column_name,column_type,null,key,default,extra
0,zip_code_prefix,VARCHAR,YES,None,None,None
1,latitude,DOUBLE,YES,None,None,None
2,longitude,DOUBLE,YES,None,None,None
3,state,VARCHAR,YES,None,None,None


In [53]:
# Sanidade do domínio (Range lat/lng)
con.execute("""
SELECT
  MIN(latitude) AS min_lat,
  MAX(latitude) AS max_lat,
  MIN(longitude) AS min_lng,
  MAX(longitude) AS max_lng
FROM curated_geolocation_by_zip;
""").df()

,min_lat,max_lat,min_lng,max_lng
0,-33.689948,4.478955,-72.916069,-32.4141


In [54]:
# Integridade (unicidade / duplicidade)
con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT zip_code_prefix) AS distinct_zip,
  COUNT(*) - COUNT(DISTINCT zip_code_prefix) AS dup_zip
FROM curated_geolocation_by_zip;
""").df()

,rows,distinct_zip,dup_zip
0,19011,19011,0


# TABLE `products` e TABLE `category_translation`

In [55]:
# Inspeção do schema (SQL)
con.execute("DESCRIBE products;").df()

,column_name,column_type,null,key,default,extra
0,product_id,VARCHAR,YES,None,None,None
1,product_category_name,VARCHAR,YES,None,None,None
2,product_name_lenght,BIGINT,YES,None,None,None
3,product_description_lenght,BIGINT,YES,None,None,None
4,product_photos_qty,BIGINT,YES,None,None,None
5,product_weight_g,BIGINT,YES,None,None,None
6,product_length_cm,BIGINT,YES,None,None,None
7,product_height_cm,BIGINT,YES,None,None,None
8,product_width_cm,BIGINT,YES,None,None,None


In [56]:
# Inspeção do schema (SQL)
con.execute("DESCRIBE category_translation;").df()

,column_name,column_type,null,key,default,extra
0,product_category_name,VARCHAR,YES,None,None,None
1,product_category_name_english,VARCHAR,YES,None,None,None


## Inspeção inicial (qualidade e sanidade)

In [57]:
# Integridade de chave ('product_id')
con.execute("""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT product_id) AS distinct_product_id,
  COUNT(*) - COUNT(DISTINCT product_id) AS dup_product_id
FROM products;
""").df()

,total_rows,distinct_product_id,dup_product_id
0,32951,32951,0


In [58]:
# Nulos críticos
con.execute("""
SELECT
  SUM(product_category_name IS NULL) AS null_category,
  SUM(product_weight_g IS NULL) AS null_weight,
  SUM(product_length_cm IS NULL) AS null_length,
  SUM(product_height_cm IS NULL) AS null_height,
  SUM(product_width_cm IS NULL) AS null_width
FROM products;
""").df()


,null_category,null_weight,null_length,null_height,null_width
0,610.0,2.0,2.0,2.0,2.0


In [59]:
# Sanidade do domínio
con.execute("""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT product_category_name) AS distinct_pt,
  COUNT(*) - COUNT(DISTINCT product_category_name) AS dup_pt,
  SUM(product_category_name IS NULL) AS null_pt,
  SUM(product_category_name_english IS NULL) AS null_en
FROM category_translation;
""").df()

,total_rows,distinct_pt,dup_pt,null_pt,null_en
0,71,71,0,0.0,0.0


## Criar view `curated_category_translation`

In [60]:
con.execute("""
CREATE OR REPLACE VIEW curated_category_translation AS
SELECT
  LOWER(TRIM(product_category_name)) AS product_category_pt,
  LOWER(TRIM(product_category_name_english)) AS product_category_en
FROM category_translation;
""")

## Criar view `curated_products`

In [61]:
con.execute("""
CREATE OR REPLACE VIEW curated_products AS
SELECT
  TRIM(p.product_id) AS product_id,

  LOWER(TRIM(p.product_category_name)) AS product_category_pt,
  COALESCE(t.product_category_en, 'unknown') AS product_category_en,

  CAST(p.product_name_lenght AS BIGINT) AS product_name_length,
  CAST(p.product_description_lenght AS BIGINT) AS product_description_length,
  CAST(p.product_photos_qty AS BIGINT) AS product_photos_qty,

  CASE WHEN p.product_weight_g  IS NULL THEN NULL WHEN p.product_weight_g  < 0 THEN 0 ELSE p.product_weight_g  END AS product_weight_g,
  CASE WHEN p.product_length_cm IS NULL THEN NULL WHEN p.product_length_cm < 0 THEN 0 ELSE p.product_length_cm END AS product_length_cm,
  CASE WHEN p.product_height_cm IS NULL THEN NULL WHEN p.product_height_cm < 0 THEN 0 ELSE p.product_height_cm END AS product_height_cm,
  CASE WHEN p.product_width_cm  IS NULL THEN NULL WHEN p.product_width_cm  < 0 THEN 0 ELSE p.product_width_cm  END AS product_width_cm

FROM products p
LEFT JOIN curated_category_translation t
  ON LOWER(TRIM(p.product_category_name)) = t.product_category_pt;
""")

### Validar `curated_produts`

In [62]:
# Schema view
con.execute("DESCRIBE curated_products;").df()

,column_name,column_type,null,key,default,extra
0,product_id,VARCHAR,YES,None,None,None
1,product_category_pt,VARCHAR,YES,None,None,None
2,product_category_en,VARCHAR,YES,None,None,None
3,product_name_length,BIGINT,YES,None,None,None
4,product_description_length,BIGINT,YES,None,None,None
5,product_photos_qty,BIGINT,YES,None,None,None
6,product_weight_g,BIGINT,YES,None,None,None
7,product_length_cm,BIGINT,YES,None,None,None
8,product_height_cm,BIGINT,YES,None,None,None
9,product_width_cm,BIGINT,YES,None,None,None


In [63]:
# Sanidade do domínio (Cobertura de categoria EN)
con.execute("""
SELECT
  COUNT(*) AS total_products,
  SUM(CASE WHEN product_category_en = 'unknown' THEN 1 ELSE 0 END) AS unknown_category,
  ROUND(100.0 * SUM(CASE WHEN product_category_en = 'unknown' THEN 1 ELSE 0 END) / COUNT(*), 2) AS pct_unknown
FROM curated_products;
""").df()


,total_products,unknown_category,pct_unknown
0,32951,623.0,1.89


In [64]:
# Sanidade do domínio (Top categorias)
con.execute("""
SELECT product_category_en, COUNT(*) AS n
FROM curated_products
GROUP BY 1
ORDER BY n DESC
LIMIT 15;
""").df()

,product_category_en,n
0,bed_bath_table,3029
1,sports_leisure,2867
2,furniture_decor,2657
3,health_beauty,2444
4,housewares,2335
5,auto,1900
6,computers_accessories,1639
7,toys,1411
8,watches_gifts,1329
9,telephony,1134


# TABLE `sellers`

In [65]:
# Inspeção do schema (SQL)
con.execute("DESCRIBE sellers;").df()

,column_name,column_type,null,key,default,extra
0,seller_id,VARCHAR,YES,None,None,None
1,seller_zip_code_prefix,VARCHAR,YES,None,None,None
2,seller_city,VARCHAR,YES,None,None,None
3,seller_state,VARCHAR,YES,None,None,None


## Inspeção inicial (qualidade e sanidade)

In [66]:
# Integridade de chave
con.execute("""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT seller_id) AS distinct_seller_id,
  COUNT(*) - COUNT(DISTINCT seller_id) AS dup_seller_id
FROM sellers;
""").df()

,total_rows,distinct_seller_id,dup_seller_id
0,3095,3095,0


In [67]:
# Nulos críticos
con.execute("""
SELECT
  SUM(seller_id IS NULL) AS null_seller_id,
  SUM(seller_zip_code_prefix IS NULL) AS null_zip,
  SUM(seller_city IS NULL) AS null_city,
  SUM(seller_state IS NULL) AS null_state
FROM sellers;
""").df()

,null_seller_id,null_zip,null_city,null_state
0,0.0,0.0,0.0,0.0


In [68]:
# Sanidade do domínio (Distribuição por UF)
con.execute("""
SELECT seller_state, COUNT(*) AS n
FROM sellers
GROUP BY 1
ORDER BY n DESC;
""").df()

,seller_state,n
0,SP,1849
1,PR,349
2,MG,244
3,SC,190
4,RJ,171
5,RS,129
6,GO,40
7,DF,30
8,ES,23
9,BA,19


## Criar view `curated_sellers`

In [69]:
con.execute("""
CREATE OR REPLACE VIEW curated_sellers AS
SELECT
  TRIM(seller_id) AS seller_id,
  TRIM(seller_zip_code_prefix) AS seller_zip_code_prefix,
  LOWER(TRIM(seller_city)) AS seller_city,
  UPPER(TRIM(seller_state)) AS seller_state
FROM sellers;
""")

### Validar `curated_sellers`

In [70]:
# Schema view
con.execute("DESCRIBE curated_sellers;").df()

,column_name,column_type,null,key,default,extra
0,seller_id,VARCHAR,YES,None,None,None
1,seller_zip_code_prefix,VARCHAR,YES,None,None,None
2,seller_city,VARCHAR,YES,None,None,None
3,seller_state,VARCHAR,YES,None,None,None


In [71]:
# Estatísticas/sanity (UF inválida)
con.execute("""
SELECT
  COUNT(*) AS total_rows,
  SUM(CASE WHEN seller_state IS NULL THEN 1 ELSE 0 END) AS null_state,
  SUM(CASE WHEN LENGTH(seller_state) <> 2 THEN 1 ELSE 0 END) AS invalid_len_state
FROM curated_sellers;
""").df()

,total_rows,null_state,invalid_len_state
0,3095,0.0,0.0


# TABLE `orders_reviews`

In [72]:
# Inspeção do schema (SQL)
con.execute("DESCRIBE order_reviews;").df()

,column_name,column_type,null,key,default,extra
0,review_id,VARCHAR,YES,None,None,None
1,order_id,VARCHAR,YES,None,None,None
2,review_score,BIGINT,YES,None,None,None
3,review_comment_title,VARCHAR,YES,None,None,None
4,review_comment_message,VARCHAR,YES,None,None,None
5,review_creation_date,TIMESTAMP,YES,None,None,None
6,review_answer_timestamp,TIMESTAMP,YES,None,None,None


## Inspeção inicial (qualidade e sanidade)

In [73]:
# Integridade de chave
con.execute("""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT review_id) AS distinct_review_id,
  COUNT(*) - COUNT(DISTINCT review_id) AS dup_review_id,
  COUNT(DISTINCT order_id) AS distinct_order_id
FROM order_reviews;
""").df()

,total_rows,distinct_review_id,dup_review_id,distinct_order_id
0,99224,98410,814,98673


In [74]:
# Nulos críticos (Distribuição de score)
con.execute("""
SELECT
  SUM(review_id IS NULL) AS null_review_id,
  SUM(order_id IS NULL) AS null_order_id,
  SUM(review_score IS NULL) AS null_score,
  MIN(review_score) AS min_score,
  MAX(review_score) AS max_score
FROM order_reviews;
""").df()

,null_review_id,null_order_id,null_score,min_score,max_score
0,0.0,0.0,0.0,1,5


In [75]:
# Um pedido pode ter mais de um review?
con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT order_id) AS distinct_orders,
  COUNT(*) - COUNT(DISTINCT order_id) AS extra_reviews_over_orders
FROM order_reviews;
""").df()

,rows,distinct_orders,extra_reviews_over_orders
0,99224,98673,551


## Criar view `curated_order_reviews`

In [76]:
con.execute("""
CREATE OR REPLACE VIEW curated_order_reviews AS
SELECT
  TRIM(review_id) AS review_id,
  TRIM(order_id) AS order_id,
  CAST(review_score AS BIGINT) AS review_score,

  review_comment_title,
  review_comment_message,

  review_creation_date AS review_creation_ts,
  review_answer_timestamp AS review_answer_ts,

  CAST(review_creation_date AS DATE) AS review_date,
  CAST(review_answer_timestamp AS DATE) AS answer_date,

  DATE_DIFF('day',
    CAST(review_creation_date AS DATE),
    CAST(review_answer_timestamp AS DATE)
  ) AS days_to_answer,

  CASE
    WHEN review_comment_message IS NULL
      OR LENGTH(TRIM(review_comment_message)) = 0
    THEN 0 ELSE 1
  END AS has_comment,

  CASE WHEN review_score IN (1,2) THEN 1 ELSE 0 END AS is_detractor,
  CASE WHEN review_score IN (4,5) THEN 1 ELSE 0 END AS is_promoter

FROM order_reviews;
""")

## Criar view `curated_reviews_by_order`

In [77]:
con.execute("""
CREATE OR REPLACE VIEW curated_reviews_by_order AS
WITH ranked AS (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY order_id
      ORDER BY review_creation_ts DESC, review_id
    ) AS rn
  FROM curated_order_reviews
)
SELECT
  order_id,
  review_id,
  review_score,
  has_comment,
  is_detractor,
  is_promoter,
  review_creation_ts,
  review_answer_ts,
  review_date,
  answer_date,
  days_to_answer
FROM ranked
WHERE rn = 1;
""")

### Validar `curated_reviews_by_order`

In [78]:
# 3 Integridade (unicidade / duplicidade) por 'order_id'
con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT order_id) AS distinct_orders,
  COUNT(*) - COUNT(DISTINCT order_id) AS dup_orders
FROM curated_reviews_by_order;
""").df()

,rows,distinct_orders,dup_orders
0,98673,98673,0


In [79]:
# Distribuição de score
con.execute("""
SELECT review_score, COUNT(*) AS n
FROM curated_reviews_by_order
GROUP BY 1
ORDER BY review_score;
""").df()

,review_score,n
0,1,11364
1,2,3127
2,3,8131
3,4,19045
4,5,57006


In [80]:
# % com comentário e tempo médio de resposta
con.execute("""
SELECT
  AVG(has_comment) AS pct_has_comment,
  AVG(days_to_answer) AS avg_days_to_answer
FROM curated_reviews_by_order;
""").df()

,pct_has_comment,avg_days_to_answer
0,0.413132,2.582854


# Estado Final do Ambiente Analítico

Visão final do banco de dados após a execução completa do processo de curadoria, permitindo verificar as tabelas e as views analíticas disponíveis.

In [81]:
# listar tabelas e views
con.execute("""
SELECT table_schema, table_name, table_type
FROM information_schema.tables
WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
ORDER BY table_type, table_schema, table_name;
""").df()

,table_schema,table_name,table_type
0,main,category_translation,BASE TABLE
1,main,customers,BASE TABLE
2,main,geolocation,BASE TABLE
3,main,order_items,BASE TABLE
4,main,order_payments,BASE TABLE
5,main,order_reviews,BASE TABLE
6,main,orders,BASE TABLE
7,main,products,BASE TABLE
8,main,sellers,BASE TABLE
9,main,curated_category_translation,VIEW


In [82]:
con.close()

# Conclusão da Curadoria SQL

Este notebook documenta o processo completo de curadoria dos dados de e-commerce
utilizados nas POCs deste repositório.

### O que foi feito
- Padronização de nomes e tipos
- Validação de integridade (unicidade, nulos, domínio)
- Criação de views analíticas reutilizáveis (`curated_*`)
- Deduplicação e agregações 1-linha-por-pedido
- Estruturação de uma camada Silver pronta para BI e Analytics

### Resultado
- Base consistente para múltiplas POCs
- Separação clara entre **curadoria (Silver)** e **análises específicas (Gold)**
- Notebook reprodutível e auditável